<a href="https://colab.research.google.com/github/akalyanam3/LLMagicians/blob/main/BaselineEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install --upgrade transformers

In [4]:
from huggingface_hub import notebook_login

notebook_login()


In [5]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Llama-3.1-8B")

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
!pip install datasets

from datasets import load_dataset

ds = load_dataset("MU-NLPC/Calc-mawps", "default")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

README.md:   0%|          | 0.00/5.88k [00:00<?, ?B/s]

(…)-00000-of-00001-4bb1451333aad61c.parquet:   0%|          | 0.00/135k [00:00<?, ?B/s]

(…)-00000-of-00001-2ce28573971ca59f.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

(…)-00000-of-00001-5a59f3fc4b0d9c98.parquet:   0%|          | 0.00/65.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1089 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1040 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/520 [00:00<?, ? examples/s]

In [10]:
test_split = ds["test"][:]
fields = list(test_split)


['id', 'question', 'chain', 'result', 'result_float', 'equation', 'expression']


In [35]:
print(pipe("Clarissa had 283 Bananas. He distributes all Bananas evenly among some friends. How many friends were there, If each of his friend get 30 Bananas? The answer to this question is:", max_length = 300))



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'Clarissa had 283 Bananas. He distributes all Bananas evenly among some friends. How many friends were there, If each of his friend get 30 Bananas? The answer to this question is: 9. Now, if he distributes the Bananas to his friends, then each of his friends will get 30 Bananas. But, if he distributes the Bananas to his friends, then each of his friends will get 30 Bananas. But, if he distributes the Bananas to his friends, then each of his friends will get 30 Bananas. But, if he distributes the Bananas to his friends, then each of his friends will get 30 Bananas. But, if he distributes the Bananas to his friends, then each of his friends will get 30 Bananas. But, if he distributes the Bananas to his friends, then each of his friends will get 30 Bananas. But, if he distributes the Bananas to his friends, then each of his friends will get 30 Bananas. But, if he distributes the Bananas to his friends, then each of his friends will get 30 Bananas. But, if he distribute

In [36]:
import time
time_arr = []
answer_arr = []
int_answer_arr = []
for i in range(len(test_split['id'])):
  prompt = test_split['question'][i] + " The answer to this question is:"

  start_time = time.time()
  generated_text = pipe(prompt)[0]['generated_text']
  end_time = time.time()

  time_arr.append(end_time - start_time)

  value = generated_text[len(prompt):].split()
  try:
    answer = eval(value[0])
    true_val = test_split['result_float'][i]
    if answer >= true_val - 0.1 and answer <= true_val + 0.1:
      answer_arr.append(1)
    else:
      answer_arr.append(0)
    if int(answer) == int(true_val):
      int_answer_arr.append(1)
    else:
      int_answer_arr.append(0)
  except:
    answer_arr.append(0)
    int_answer_arr.append(0)






Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [38]:
print(time_arr)
print(answer_arr)
print(int_answer_arr)

print(len(time_arr))
print(len(answer_arr))
print(len(int_answer_arr))



[0.8053531646728516, 0.787766695022583, 0.7850368022918701, 0.7850189208984375, 0.7861218452453613, 0.749889612197876, 0.779202938079834, 0.7926905155181885, 0.7904167175292969, 0.7565865516662598, 0.5234313011169434, 0.785344123840332, 0.760573148727417, 0.7871263027191162, 0.7868869304656982, 0.7583684921264648, 0.7852232456207275, 0.7837400436401367, 0.7866158485412598, 0.785942792892456, 0.7807762622833252, 0.7842841148376465, 0.7827639579772949, 0.7784590721130371, 0.7745361328125, 0.7508759498596191, 0.7498219013214111, 0.7737770080566406, 0.7465841770172119, 0.7769675254821777, 0.592449426651001, 0.7803909778594971, 0.7684512138366699, 0.7797853946685791, 0.7454075813293457, 0.7077398300170898, 0.7860791683197021, 0.7934913635253906, 0.7566871643066406, 0.7846024036407471, 0.7341020107269287, 0.7474839687347412, 0.18940091133117676, 0.7738285064697266, 0.7475855350494385, 0.7799997329711914, 0.7808635234832764, 0.7477960586547852, 0.7790215015411377, 0.7478318214416504, 0.771937

In [47]:
answer_sum = 0
for answer in answer_arr:
  answer_sum += answer
print(answer_sum)

int_answer_sum = 0
for answer in int_answer_arr:
  int_answer_sum += answer
print(int_answer_sum)

time_sum = 0
for time in time_arr:
  time_sum += time
avg_time_ms = time_sum/520 * 1000


299
351


In [46]:
print(str(answer_sum*100/520) + "% accuracy rate when we consider integer responses for fractional/decimal answers as invalid.")
print(str(int_answer_sum*100/520) + "% accuracy rate when we consider integer responses for fractional/decimal answers as valid.")
print(str(avg_time_ms) + " ms average reasoning time, including pipeline connection time.")

57.5% accuracy rate when we consider integer responses for fractional/decimal answers as invalid.
67.5% accuracy rate when we consider integer responses for fractional/decimal answers as valid.
729.6377700108749 ms average reasoning time, including pipeline connection time.
